# MNIST 手寫數字辨識 (MNIST_DNN_TF)

2017/07/16 update tensorboard 
visualization  
徐仕杰

### Tips:
- 記得要download data set: 
[Mnist](https://github.com/Backlu/tf-keras-tutorial/blob/master/basic/mnist.pkl.xz)
- 在command前面加** ! **可以執行console command
- 在command前面加** ? **可以查詢Help
- 什麼是one-hot representation:
[one-hot](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)  
- 好用的markdown語法
[markdown](https://www.zybuluo.com/codeep/note/163962#1如何输入一个方程式序列)  
<br>
- import PIL error : pip install Pillow
- import pandas error: pip install pandas
- import lzma error: 請用python 3



## Outline

-  [Import Package & Functions](#import) 
-  [1. Import MNIST Data](#Import Data) 
-  [2. seMMA-DNN](#開始Deep Learning)  
-  [3. Reference](#reference)

<a id='import'></a>
## Import Package & Functions

In [1]:
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tfdot import tfdot
from tensorflow.contrib.tensorboard.plugins import projector
import shutil

In [2]:
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))

In [3]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [4]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

<a id='Import MNIST Data'></a>
## 1. Import MNIST Data

#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [5]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])


list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [6]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [7]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='開始Deep Learning'></a>
## 2. 開始Deep Nerual Network
- What is DNN?  DNN就是很多層的f(WX+B)

**這是一層的DNN(其實就是Softmax Regression)**：
![](img/dnn_1.png)

**這是很多層的DNN**
![](img/dnn_2.png)  


- Deep Learning ABC 
    -  [A. 定義參數](#定義參數) 
    -  [B. 設計一個Model從X預測Y](#設計一個) 
    -  [C. 選一個loss function,](#選一個loss) 
    -  [D. 選一個optimizer](#選一個o) 
    -  [E. 開始執行訓練](#開始執行) 
    -  [F. 算一下正確率](#算一下正)  

<a id='定義參數'></a>
### A. 定義參數(Placeholder, Variable, Constant)
tips: 把要餵進Model的資料X,Y定義成placeholder, 把要讓電腦幫忙找的權重W,B定義成Variable

In [8]:
# hyperparameters
lr = 0.01 # learning rate
n_inputs = 784 # 每一行的维度
n_classes = 10  # RNN最后的输出類別個数

In [9]:
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_inputs], name="X")
Y_ =tf.placeholder(tf.float32, [None, n_classes], name="Y_")

W = {
    'wd1': tf.Variable(tf.random_normal([784,600], stddev=0.01), name="wd1"),
    'wd2': tf.Variable(tf.random_normal([600,480], stddev=0.01), name="wd2"),
    'out': tf.Variable(tf.random_normal([480, 10]), name="out")
}
variable_summaries(W['wd1'],'wd1')
variable_summaries(W['wd2'],'wd2')
variable_summaries(W['out'],'out')

B = {
    'bd1': tf.Variable(tf.random_normal([600]),name="bd1"),
    'bd2': tf.Variable(tf.random_normal([480]), name="bd2"),
    'out': tf.Variable(tf.random_normal([10]), name="out"),
}

variable_summaries(B['bd1'],'bd1')
variable_summaries(B['bd2'],'bd2')
variable_summaries(B['out'],'out')

#tfdot() 不看!

<a id='設計一個'></a>
###  B. 設計一個Model從X & X history預測Y  
Input Layer: X  
Hidden Layer 1: $H_1=f_1(W_1X+B_1)$  
Hidden Layer 2: $H_2=f_2(W_2H_1+B_2)$  
Output Layer : $Y=f_3(W_3H_2+B_3)$  


### L1. Input Layer 輸入層
- do nothing

### L2. Hidden Layer 隱藏層 x 2
- 線性(WX+B) + 非線性(activation function)  
![](img/activation.jpg)

In [10]:
with tf.name_scope("Hidden_Layer1"):
    _H1 = tf.matmul(X, W['wd1']) + B['bd1'] 
    H1 = tf.nn.relu(_H1, name="H1")

In [11]:
with tf.name_scope("Hidden_Layer2"):
    _H2 = tf.matmul(H1, W['wd2']) + B['bd2'] # (-1, 28) matmul (28, hidden_units) => (-1, hidden_units)
    H2 = tf.nn.relu(_H2, name="H2")

### L3. Output Layer 輸出層
- 第一步：線性 (重點：把output轉換成要分類的數量)
- 第二步：非線性 (重點：把數值壓縮在0~1之間, 類似機率的表示方法)

In [12]:
with tf.name_scope('outlayer'):
    _pred = tf.matmul(H2, W['out']) + B['out']
    pred = tf.nn.softmax(_pred, name="pred")
    
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

<a id='選一個loss'></a>
###  C. 選一個loss function, 當作Machine learning的目標
- cross_entorpy $-log(\Pr(Y_{true}))$

In [13]:
#這邊要注意, loss function裡面已經有做softmax, 所以logits記得帶沒有做softmax的
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=Y_))

<a id='選一個o'></a>
### D. 選一個optimizer, 根據Data和我們訂的目標找參數W, B
- 試試看adaptive gradient descent(adagrad), 這個方法會在每個步驟都根據前面步驟的梯度來調整learing rate, 大致上的概念就是一開始走快一點, 接近最低點的時候走小步一點, 當梯度值突然很大的時候也走大步一點, 看一下李宏毅教授的上課影片就能理解    
<br>
$W^{t+1}=W^t- {\frac{\eta^t}{\sigma^t}}g^t$  
$\sigma=\sqrt{\frac{1}{t+1}\sum({g^i})^2}$
<br>  
$\eta$: learning rate  
$\sigma$: 過去所有梯度的root mean square  
$g$: 梯度值  


In [14]:
optimizer = tf.train.AdagradOptimizer(lr).minimize(loss)

***

<a id='開始執行'></a>
### E. 開始執行訓練(Training Data + Validataion Data)

In [15]:
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
with tf.name_scope('performance'):  
    loss_scar = tf.summary.scalar('loss', loss)
    # Create a summary to monitor accuracy tensor
    acc_scar = tf.summary.scalar('accuracy', accuracy)
    # Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [16]:
#先創造一個session, 然後記得要init variable
sess = tf.InteractiveSession()
sess.run(init)
LOG_DIR = 'tb-log5'
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
summary_writer_train = tf.summary.FileWriter(LOG_DIR+'/train',graph=tf.get_default_graph())
summary_writer_validation = tf.summary.FileWriter(LOG_DIR+'/validation',graph=tf.get_default_graph())

In [17]:
epoch = 15
batch_size = 128
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X.shape[0], batch_size, replace=False)
        batch_x = train_X[rnd_idx]
        batch_y = train_Y[rnd_idx]
        _, loss_s, acc_s, summary= sess.run([optimizer, loss_scar, acc_scar, merged_summary_op], feed_dict={X: batch_x, Y_:batch_y})
        summary_writer_train.add_summary(summary, ep * total_batch + i)
        if i%100 ==0:
            loss_s, acc_s, summary= sess.run([loss_scar, acc_scar, merged_summary_op], feed_dict={X: validation_X , Y_: validation_Y})
            acc = accuracy.eval({X: validation_X , Y_: validation_Y})
            los=loss.eval({X: validation_X , Y_: validation_Y})
            summary_writer_validation.add_summary(summary, ep * total_batch + i)
            updateProgress('epoch:{x0}, batch:{x4} loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc,x3=los,x4=i))
    print()


epoch:0, batch:300 loss:0.2885988652706146, acc:0.91469997167587282
epoch:1, batch:300 loss:0.1726723611354828, acc:0.95139998197555548
epoch:2, batch:300 loss:0.14686638116836548, acc:0.9577000141143799
epoch:3, batch:300 loss:0.12102670967578888, acc:0.9663000106811523
epoch:4, batch:300 loss:0.11033638566732407, acc:0.9685000181198123
epoch:5, batch:300 loss:0.10239648073911667, acc:0.9697999954223633
epoch:6, batch:300 loss:0.11012419313192368, acc:0.9679999947547913
epoch:7, batch:300 loss:0.09443458914756775, acc:0.9726999998092651
epoch:8, batch:300 loss:0.08842770755290985, acc:0.9745000004768372
epoch:9, batch:300 loss:0.09069573134183884, acc:0.9732000231742859
epoch:10, batch:300 loss:0.08624941110610962, acc:0.9754999876022339
epoch:11, batch:300 loss:0.0873347669839859, acc:0.97479999065399178
epoch:12, batch:300 loss:0.08525534719228745, acc:0.9740999937057495
epoch:13, batch:300 loss:0.08181624114513397, acc:0.9764000177383423
epoch:14, batch:300 loss:0.08127942681312561

In [18]:
# 抓出前五筆訓練資料的預測結果 Y_softmax (注意：這個是one-hot的格式)
pred_tmp = pred.eval(feed_dict={X: train_X[21:30]})
pred_tmp_df = pd.DataFrame(pred_tmp)

In [19]:

_ = pred_tmp_df.apply(np.argmax,axis=1)
print('predict: ',_.values)
showX(train_X[21:30])
print('answer: ',train_y[21:30])

_ = correct_pred.eval({X: train_X[21:30] , Y_: train_Y[21:30]})
print('第21~30筆資料的比對結果:',pd.DataFrame(_))

predict:  [0 9 1 1 2 4 3 2 7]


answer:  [0 9 1 1 2 4 3 2 7]
第21~30筆資料的比對結果:       0
0  True
1  True
2  True
3  True
4  True
5  True
6  True
7  True
8  True


<a id='算一下正'></a>
### F. 算一下正確率Accuracy (用Testing Data)
- unseen data
- generalize from the training set to the test set
- 泛化

In [20]:
#pred_test = pred.eval(feed_dict={X: test_X})
#pred_test_df = pd.DataFrame(pred_test)
#pred_test_df = pred_test_df.apply(np.argmax,axis=1)
# predict all X and get the accuracy
acc = sess.run(accuracy, feed_dict={X: test_X , Y_: test_Y})
print('Accuracy:',acc)

Accuracy: 0.9752


## Visualization

In [21]:
#把圖片的label存到metadata.tsv
metadata_file_path = os.path.join(LOG_DIR, 'metadata.tsv')
with open(metadata_file_path, 'w') as metadata_file:
    for row in range(len(test_y)):
        c=test_y[row]
        metadata_file.write('{}\n'.format(c))

In [22]:
#Save variable
Input_Var = tf.Variable(test_X, name='Input_var')
H1_Var = tf.Variable(H1.eval({X: test_X}), name='H1_var')
H2_Var = tf.Variable(H2.eval({X: test_X}), name='H2_var')
Out_Var = tf.Variable(pred.eval({X: test_X}), name='Out_var')

saver = tf.train.Saver([Input_Var,H1_Var,H2_Var,Out_Var])
sess.run(Input_Var.initializer)
sess.run(H1_Var.initializer)
sess.run(H2_Var.initializer)
sess.run(Out_Var.initializer)
saver.save(sess, os.path.join(LOG_DIR, 'test_images.ckpt'))

#在config裡面用一個embedding關聯 tensor & its metadata
config = projector.ProjectorConfig()
# One can add multiple embeddings.

embedding1 = config.embeddings.add()
embedding1.tensor_name = Input_Var.name
embedding1.metadata_path = metadata_file_path

embedding2 = config.embeddings.add()
embedding2.tensor_name = H1_Var.name
embedding2.metadata_path = metadata_file_path

embedding3 = config.embeddings.add()
embedding3.tensor_name = H2_Var.name
embedding3.metadata_path = metadata_file_path

embedding4 = config.embeddings.add()
embedding4.tensor_name = Out_Var.name
embedding4.metadata_path = metadata_file_path

#embedding.sprite.image_path = os.path.join(LOG_DIR, 'img/mnist_10k_sprite.png')
# Specify the width and height of a single thumbnail.
#embedding.sprite.single_image_dim.extend([28, 28])



# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


![](img/mnist001.png)

In [23]:
#!tensorboard --logdir=tb-log1 --port=6006

In [24]:
#sess.close()

### Homework 
- 玩玩看CIFAR 10 Data set
- 李宏毅教授上課影片   
[Regression](https://www.youtube.com/watch?v=fegAeph9UaA) : ML 101  
[Where does the error come from](https://www.youtube.com/watch?v=D_S6y0Jm6dQ) : Bias, Variance     
[Gradient Descent](https://www.youtube.com/watch?v=yKKNr-QKz2Q) : some useful tips   

<a id='reference'></a>
## Reference
- https://my.oschina.net/yilian/blog/664077

### 補充資料
![](img/dmap.jpeg)

![](img/one_hot.jpg)

In [25]:
save_model=True
restore_model = True

In [26]:
if save_model:
    saver = tf.train.Saver()
    save_path = saver.save(sess, "/tmp/dnn_model.ckpt")
    print('save dnn model done',save_path)

save dnn model done /tmp/dnn_model.ckpt


In [27]:
if restore_model:
    sess=tf.Session()
    saver = tf.train.Saver()
    saver.restore(sess, "/tmp/dnn_model.ckpt")
    print('restore dnn model done')

INFO:tensorflow:Restoring parameters from /tmp/dnn_model.ckpt
restore dnn model done
